In [1]:
# %pip install snowflake-connector-python
# %pip install pyarrow==8.0.0
# %pip install pandas
# %pip install confluent-kafka
# %pip install python-dotenv

In [2]:
import snowflake.connector
from snowflake_helpers import *
import re
import pandas as pd
from transformation_helpers import *

In [3]:
conn = connect_to_snowflake()
cs = conn.cursor()

Connected to Snowflake with user: admin


In [4]:
create_production_schema(cs)
use_production_schema(cs)

Schema: yusra_stories_production created.


In [5]:
df = cs.execute('SELECT * FROM yusra_stories_staging.logs').fetch_pandas_all()
df.head(5)

,log_id,log
0,1,ride 1 --------- beginning of a new ride\n
1,2,ride 1 2022-10-06 15:49:21.453226 mendoza v9: ...
2,3,ride 1 2022-10-06 15:49:21.953902 mendoza v9: ...
3,4,ride 1 2022-10-06 15:49:22.454586 mendoza v9: ...
4,5,ride 1 2022-10-06 15:49:22.955260 mendoza v9: ...


In [6]:
formatted_df = get_joined_formatted_df(df)

at the start df is <class 'pandas.core.frame.DataFrame'>


/Users/harrymay/Desktop/coursework/deloton-group-project/transformation_helpers.py:62: FutureWarning: Comparison of NaT with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable.
  if birthday > today:
/Users/harrymay/Desktop/coursework/deloton-group-project/transformation_helpers.py:62: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if birthday > today:


In [7]:
formatted_df.head(3)

,log_id,log,ride_id,is_new_ride,is_info,is_system,time,user_id,name,gender,...,address,email_address,account_created,bike_serial,original_source,ride_duration_secs,heart_rate,resistance,rpm,power
0,1,ride 1 --------- beginning of a new ride\n,1,True,False,False,NaT,NaN,None,None,...,None,None,NaT,None,None,NaN,NaN,NaN,NaN,NaN
1,2,ride 1 2022-10-06 15:49:21.453226 mendoza v9: ...,1,False,False,False,2022-10-06 15:49:21.453226,NaN,None,None,...,None,None,NaT,None,None,NaN,NaN,NaN,NaN,NaN
2,3,ride 1 2022-10-06 15:49:21.953902 mendoza v9: ...,1,False,False,True,2022-10-06 15:49:21.953902,4612.0,Mrs Bethany Welch,female,...,"Flat 3,Jeremy rapid,Evanshaven,S2 9DH",mrs_b@gmail.com,2022-02-27,SN0000,direct,NaN,NaN,NaN,NaN,NaN


In [8]:
user_df = get_users_df(formatted_df)
user_df.head()

,name,gender,date_of_birth,age,height_cm,weight_kg,address,email_address,account_created,bike_serial,original_source
user_id,,,,,,,,,,,
4612.0,Mrs Bethany Welch,female,1968-09-20,54.0,188.0,60.0,"Flat 3,Jeremy rapid,Evanshaven,S2 9DH",mrs_b@gmail.com,2022-02-27,SN0000,direct
4614.0,Jill Gill,female,1996-11-11,25.0,180.0,61.0,"Studio 4,Carly valleys,West George,RH72 6JL",j_g86@gmail.com,2022-03-02,SN0000,google ads


For future table insert

In [9]:
# cs.execute(f"""
#         CREATE OR REPLACE TABLE df_test_table
#         ("log" STRING,
#         "is_new_ride" BOOLEAN,
#         "is_info" BOOLEAN,
#         "is_system" BOOLEAN,
#         "date" DATETIME,
#         "heart_rate" INTEGER,
#         "resistance" INTEGER,
#         "rpm" INTEGER,
#         "power" FLOAT,
#         "user_id" STRING,
#         "name" STRING,
#         "gender" STRING,
#         "address" STRING,
#         "date_of_birth" FLOAT,
#         "email_address" STRING,
#         "height_cm" INTEGER,
#         "weight_kg" INTEGER,
#         "account_create_date" FLOAT,
#         "bike_serial" STRING,
#         "original_source" STRING
#         )
# """)

In [10]:
# write_pandas(conn, df, 'DF_TEST_TABLE')

In [11]:
# cs.execute(f'SELECT * FROM DF_TEST_TABLE WHERE "is_system" = TRUE').fetch_pandas_all()